# Jacobi Iterative Method for Solving Linear Systems

## Introduction

The **Jacobi iterative method** is a classical algorithm for solving systems of linear equations of the form:

$$A\mathbf{x} = \mathbf{b}$$

where $A \in \mathbb{R}^{n \times n}$ is a square matrix, $\mathbf{x} \in \mathbb{R}^n$ is the unknown solution vector, and $\mathbf{b} \in \mathbb{R}^n$ is the right-hand side vector.

## Mathematical Foundation

### Matrix Decomposition

The Jacobi method is based on decomposing the matrix $A$ into its diagonal and off-diagonal components:

$$A = D + L + U$$

where:
- $D$ is the diagonal matrix containing the diagonal elements of $A$
- $L$ is the strictly lower triangular part of $A$
- $U$ is the strictly upper triangular part of $A$

### Iterative Formula

Starting from the equation $A\mathbf{x} = \mathbf{b}$, we can write:

$$(D + L + U)\mathbf{x} = \mathbf{b}$$

Rearranging:

$$D\mathbf{x} = \mathbf{b} - (L + U)\mathbf{x}$$

This leads to the Jacobi iteration formula:

$$\mathbf{x}^{(k+1)} = D^{-1}\left(\mathbf{b} - (L + U)\mathbf{x}^{(k)}\right)$$

### Component-wise Form

For each component $i = 1, 2, \ldots, n$:

$$x_i^{(k+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{j \neq i} a_{ij} x_j^{(k)}\right)$$

### Convergence Conditions

The Jacobi method converges if:

1. **Strict diagonal dominance**: $|a_{ii}| > \sum_{j \neq i} |a_{ij}|$ for all $i$

2. **Spectral radius condition**: $\rho(D^{-1}(L+U)) < 1$

The rate of convergence depends on the spectral radius of the iteration matrix $G = D^{-1}(L+U)$.

### Error Analysis

The error at iteration $k$ satisfies:

$$\|\mathbf{e}^{(k)}\| \leq \|G\|^k \|\mathbf{e}^{(0)}\|$$

where $\mathbf{e}^{(k)} = \mathbf{x}^{(k)} - \mathbf{x}^*$ is the error vector.

## Implementation

Let us implement the Jacobi iterative method and demonstrate its convergence properties.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, eigvals

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
def jacobi_iteration(A, b, x0=None, tol=1e-10, max_iter=1000):
    """
    Solve Ax = b using the Jacobi iterative method.
    
    Parameters:
    -----------
    A : ndarray
        Coefficient matrix (n x n)
    b : ndarray
        Right-hand side vector (n,)
    x0 : ndarray, optional
        Initial guess (default: zeros)
    tol : float
        Convergence tolerance
    max_iter : int
        Maximum number of iterations
        
    Returns:
    --------
    x : ndarray
        Solution vector
    history : dict
        Convergence history containing errors and residuals
    """
    n = len(b)
    
    # Initial guess
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0.copy()
    
    # Extract diagonal
    D = np.diag(A)
    
    # Check for zero diagonal elements
    if np.any(D == 0):
        raise ValueError("Matrix has zero diagonal elements")
    
    # History tracking
    history = {
        'iterations': [],
        'residuals': [],
        'errors': [],
        'solutions': [x.copy()]
    }
    
    # Compute exact solution for error tracking
    x_exact = np.linalg.solve(A, b)
    
    for k in range(max_iter):
        x_new = np.zeros(n)
        
        # Jacobi iteration
        for i in range(n):
            sigma = np.dot(A[i, :], x) - A[i, i] * x[i]
            x_new[i] = (b[i] - sigma) / A[i, i]
        
        # Compute residual and error
        residual = norm(b - A @ x_new)
        error = norm(x_new - x_exact)
        
        # Store history
        history['iterations'].append(k + 1)
        history['residuals'].append(residual)
        history['errors'].append(error)
        history['solutions'].append(x_new.copy())
        
        # Check convergence
        if norm(x_new - x) < tol:
            return x_new, history
        
        x = x_new
    
    print(f"Warning: Maximum iterations ({max_iter}) reached")
    return x, history

In [ ]:
def check_diagonal_dominance(A):
    """
    Check if matrix A is strictly diagonally dominant.
    """
    n = A.shape[0]
    for i in range(n):
        diag = abs(A[i, i])
        off_diag_sum = sum(abs(A[i, j]) for j in range(n) if j != i)
        if diag <= off_diag_sum:
            return False
    return True


def compute_spectral_radius(A):
    """
    Compute the spectral radius of the Jacobi iteration matrix.
    """
    D = np.diag(np.diag(A))
    L_plus_U = A - D
    G = -np.linalg.inv(D) @ L_plus_U
    eigenvalues = eigvals(G)
    return np.max(np.abs(eigenvalues))

## Example 1: Diagonally Dominant System

We construct a strictly diagonally dominant matrix to ensure convergence.

In [ ]:
# Create a diagonally dominant matrix
n = 5
A = np.random.rand(n, n)

# Make it diagonally dominant
for i in range(n):
    A[i, i] = np.sum(np.abs(A[i, :])) + 1

# Create right-hand side
b = np.random.rand(n)

print("Matrix A:")
print(A)
print("\nVector b:")
print(b)
print(f"\nStrictly diagonally dominant: {check_diagonal_dominance(A)}")
print(f"Spectral radius of iteration matrix: {compute_spectral_radius(A):.6f}")

In [ ]:
# Solve using Jacobi method
x_jacobi, history = jacobi_iteration(A, b)

# Compare with direct solution
x_exact = np.linalg.solve(A, b)

print("Jacobi solution:")
print(x_jacobi)
print("\nExact solution:")
print(x_exact)
print(f"\nNumber of iterations: {len(history['iterations'])}")
print(f"Final error: {history['errors'][-1]:.2e}")

## Example 2: Classic Tridiagonal System

Consider the classic tridiagonal system arising from discretization of the 1D Poisson equation:

$$-u''(x) = f(x)$$

This leads to a tridiagonal matrix with 2 on the diagonal and -1 on the off-diagonals.

In [ ]:
# Create tridiagonal matrix (1D Poisson)
n = 50
A_tri = np.diag(2 * np.ones(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)

# Right-hand side (constant source)
b_tri = np.ones(n)

print(f"Matrix size: {n} x {n}")
print(f"Strictly diagonally dominant: {check_diagonal_dominance(A_tri)}")
print(f"Spectral radius: {compute_spectral_radius(A_tri):.6f}")

In [ ]:
# Solve tridiagonal system
x_tri, history_tri = jacobi_iteration(A_tri, b_tri, tol=1e-8, max_iter=5000)

print(f"Converged in {len(history_tri['iterations'])} iterations")
print(f"Final residual: {history_tri['residuals'][-1]:.2e}")

## Convergence Analysis and Visualization

In [ ]:
# Create figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Convergence of error (Example 1)
ax1 = axes[0, 0]
ax1.semilogy(history['iterations'], history['errors'], 'b-', linewidth=2, marker='o', markersize=4)
ax1.set_xlabel('Iteration', fontsize=12)
ax1.set_ylabel('Error $\\|x^{(k)} - x^*\\|$', fontsize=12)
ax1.set_title('Error Convergence (Diagonally Dominant System)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=1e-10, color='r', linestyle='--', label='Tolerance')
ax1.legend()

# Plot 2: Convergence of residual (Example 1)
ax2 = axes[0, 1]
ax2.semilogy(history['iterations'], history['residuals'], 'g-', linewidth=2, marker='s', markersize=4)
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Residual $\\|b - Ax^{(k)}\\|$', fontsize=12)
ax2.set_title('Residual Convergence (Diagonally Dominant System)', fontsize=12)
ax2.grid(True, alpha=0.3)

# Plot 3: Convergence for tridiagonal system
ax3 = axes[1, 0]
ax3.semilogy(history_tri['iterations'], history_tri['errors'], 'r-', linewidth=1.5)
ax3.set_xlabel('Iteration', fontsize=12)
ax3.set_ylabel('Error $\\|x^{(k)} - x^*\\|$', fontsize=12)
ax3.set_title(f'Error Convergence (Tridiagonal System, n={n})', fontsize=12)
ax3.grid(True, alpha=0.3)

# Plot 4: Solution comparison for tridiagonal system
ax4 = axes[1, 1]
x_exact_tri = np.linalg.solve(A_tri, b_tri)
grid_points = np.linspace(0, 1, n)
ax4.plot(grid_points, x_tri, 'b-', linewidth=2, label='Jacobi Solution')
ax4.plot(grid_points, x_exact_tri, 'r--', linewidth=2, label='Exact Solution')
ax4.set_xlabel('x', fontsize=12)
ax4.set_ylabel('u(x)', fontsize=12)
ax4.set_title('Solution of 1D Poisson Equation', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Effect of Spectral Radius on Convergence Rate

Let us investigate how the spectral radius affects the convergence rate.

In [ ]:
# Create systems with different spectral radii
spectral_radii = []
iterations_needed = []

for alpha in [0.1, 0.3, 0.5, 0.7, 0.9]:
    # Create matrix with controlled spectral radius
    n_test = 10
    A_test = np.eye(n_test)
    for i in range(n_test):
        for j in range(n_test):
            if i != j:
                A_test[i, j] = alpha / (n_test - 1)
    
    b_test = np.ones(n_test)
    rho = compute_spectral_radius(A_test)
    
    _, hist = jacobi_iteration(A_test, b_test, tol=1e-8)
    
    spectral_radii.append(rho)
    iterations_needed.append(len(hist['iterations']))
    print(f"α = {alpha:.1f}: ρ = {rho:.4f}, iterations = {len(hist['iterations'])}")

## Summary

The Jacobi iterative method is a fundamental algorithm for solving linear systems with the following key properties:

1. **Simplicity**: Easy to implement and parallelize
2. **Convergence**: Guaranteed for strictly diagonally dominant matrices
3. **Convergence Rate**: Determined by the spectral radius $\rho(G)$
4. **Memory Efficiency**: Only requires storage for two vectors

**Limitations**:
- Slower convergence compared to Gauss-Seidel and SOR methods
- May not converge for all matrices
- Requires non-zero diagonal elements

The method is particularly useful as a building block for more sophisticated iterative solvers and as a smoother in multigrid methods.